<a href="https://colab.research.google.com/github/boffandre-cyber/PEE---2025/blob/main/C%C3%B3pia_de_C%C3%B3pia_de_TRB1_PEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ainda em falta:

*   ajustar os comentários durante o código (padronizar mais), escrever uma breve
*   introdução explicando objetivos,
*   colocar todas as bibliotecas que estão sendo usadas na primeira secção de código e explica-las no texto,
*   ajustar texto dos gráficos e alguns gráficos com subíndices e explicar cada parte,
*   fazer a parte de texto do reservatório
* Melhorar os atuais textos

Introdução
---
___

In [1]:
!git clone https://github.com/boffandre-cyber/PEE---2025.git

Cloning into 'PEE---2025'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 40 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 1.64 MiB | 9.74 MiB/s, done.
Resolving deltas: 100% (21/21), done.


Importando Biblitecas e suas funções
---
---
Esta secção de códogo é dedicada e importar todas as bibliotecas que serão utilizadas ao longo do projeto, sendo elas:
*   Pandas:
*   Numpy:
*   Matplotlib:



In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

Lendo o arquivo de dados
---
---
Inicialmente foi buscado um arquivo fluvial no site da ANA (Agencia Nacional das Águas) o qual está no formato csv, portanto deve-se ajusta-lo para a leitura correta pela Biblioteca Pandas, afim de fazer uma análise de dados posterior em forma de dataframe


In [ ]:
# Esta função é necessária para lidar com o formato específico do arquivo,
# que contém metadados antes do cabeçalho.
import re # Import the re module

def read_complex_csv(file_path):
    """
    Lê um arquivo CSV com metadados e retorna um DataFrame.
    """
    with open(file_path, 'r', encoding='latin-1') as f:
        lines = f.readlines()

    # O cabeçalho está na linha 16 (índice 15).
    header_line = lines[15].strip()
    data_lines = lines[16:]

    # Obter os nomes das colunas, removendo aspas duplas.
    columns = [col.strip('"') for col in header_line.split(';')]

    # Processar as linhas de dados, removendo aspas e substituindo vírgulas por pontos.
    data = []
    for line in data_lines:
        if line.strip():  # Ignorar linhas vazias
            row = [re.sub(r'"', '', val).replace(',', '.') for val in line.split(';')]
            data.append(row)

    # Criar o DataFrame a partir dos dados processados.
    df = pd.DataFrame(data, columns=columns)

    # Remover colunas totalmente vazias ou sem nome
    df = df.dropna(axis=1, how='all')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    return df

# Substitua '70100000_Vazoes_I.csv' pelo nome do arquivo.
file_path = '70100000_Vazoes_I.csv'
df = read_complex_csv(file_path)

Análise de Dados
---
___
Análise inicial de dados onde é ajustado os dados de vazões para estar em conformidade com o estipulado como premissa em edital da disciplina. Nela também são ajustadas as colunas que inicialmente são entendidas como variáveis de String, portanto são ajustadas datas na forma de DateTime e vazões como Float

In [ ]:
# -----------------------------------------------------------------------------
# Início da análise
# -----------------------------------------------------------------------------

# Converter a coluna 'Data' para o formato de data
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

# Encontrar o ano mais recente nos dados
most_recent_year = df['Data'].dt.year.max()
print(f"Ano mais recente nos dados: {most_recent_year}")

# Definir o ano de corte para desconsiderar os dois últimos anos
year_cutoff = most_recent_year - 1
print(f"Os dados serão filtrados até o ano de: {year_cutoff - 1}")

# Filtrar o DataFrame para excluir os dois últimos anos
df_filtered = df[df['Data'].dt.year < year_cutoff].copy()

# Identificar as colunas de vazão, excluindo aquelas que contêm a palavra 'Status'
vazao_cols = [col for col in df_filtered.columns if 'Vazao' in col and 'Status' not in col]
print(f"\nColunas de vazão encontradas: {vazao_cols}")

# Desempilhar o DataFrame para obter uma única coluna com todos os valores de vazão
df_melted_filtered = df_filtered.melt(value_vars=vazao_cols, value_name='Vazao')

# Converter a coluna 'Vazao' para tipo numérico e remover valores ausentes
df_melted_filtered['Vazao'] = pd.to_numeric(df_melted_filtered['Vazao'], errors='coerce')
df_melted_filtered.dropna(subset=['Vazao'], inplace=True)

# Calcular a vazão média de longo termo
long_term_avg = df_melted_filtered['Vazao'].mean()

print(f"\nVazão média de longo termo (excluindo os dois últimos anos): {long_term_avg:.2f} m³/s")

# Definir o intervalo de referência e o valor alvo
lower_bound = 30
upper_bound = 60
target_avg = 45 # Ponto central do intervalo

# Verificar se a vazão média está fora do intervalo
if not (lower_bound <= long_term_avg <= upper_bound):
    print(f"\nA vazão média de longo termo está fora do intervalo [{lower_bound}, {upper_bound}] m³/s.")
    print("Aplicando fator de correção.")

    # Calcular o fator de correção
    correction_factor = target_avg / long_term_avg
    print(f"Fator de correção calculado: {correction_factor:.4f}")

    # Criar uma cópia do DataFrame original para aplicar a correção
    df_corrected = df.copy()

    # Aplicar o fator de correção a todas as colunas de vazão
    for col in vazao_cols:
        df_corrected[col] = pd.to_numeric(df_corrected[col], errors='coerce').mul(correction_factor)

    # Recalcular a média de longo termo com os dados corrigidos para verificação
    df_melted_corrected = df_corrected.melt(value_vars=vazao_cols, value_name='Vazao')
    df_melted_corrected['Vazao'] = pd.to_numeric(df_melted_corrected['Vazao'], errors='coerce')
    corrected_avg = df_melted_corrected['Vazao'].mean()
    print(f"Nova vazão média de longo termo após correção: {corrected_avg:.2f} m³/s")

    # Salvar o DataFrame corrigido em um novo arquivo CSV
    df_corrected.to_csv('70100000_Vazoes_I_Corrigido.csv', sep=';', index=False, decimal=',')
    print("\nUm novo arquivo CSV com os dados corrigidos foi gerado e está disponível como '70100000_Vazoes_I_Corrigido.csv'.")
else:
    print(f"\nA vazão média de longo termo já está dentro do intervalo [{lower_bound}, {upper_bound}] m³/s. Nenhuma correção é necessária.")


# Criando uma coluna de mediana:
#df_corrected['Mediana'] = df_corrected[vazao_cols].median(axis=1)

# Ajustantdo as colunas de media minima e maxima do df
df_corrected['Media'] = pd.to_numeric(df_corrected['Media'], errors='coerce')
df_corrected['Minima'] = pd.to_numeric(df_corrected['Minima'], errors='coerce')
df_corrected['Maxima'] = pd.to_numeric(df_corrected['Maxima'], errors='coerce')



for col in ['Media', 'Minima', 'Maxima']:
    df_corrected[col] = df_corrected[col] * correction_factor

# Criando uma coluna de mediana:
df_corrected['Mediana'] = df_corrected[vazao_cols].median(axis=1)

Plotagem dos Gráficos
---
___
Aqui são plotados os gráficos de vazões para o histórico de 40 anos. São feitos os gráficos da vazão média mensal e curva de carga para todo o histórico.
Também nesta secção se encontra a criação de um ano sintético o qual foi utilizado para fazer os gráficos das médias mensais e de Rippl.

In [ ]:
#grafico de vazao da média mensal de todos os anos
df_corrected


#plotando todos os valores de média
df_corrected.plot(x='Data', y='Media', kind='line')
plt.title('Vazão ao longo dos anos')
plt.xlabel('data')
plt.ylabel('vazão média')
plt.grid(True)
plt.show()

#aqui plotar sazonalidades fazer media das medias, das minimas e das maximas dos meses.
df_anosintetico = df_corrected
# Extrai o mês da data
df_anosintetico['Mes'] = df_anosintetico['Data'].dt.month

# Agrupa por mês e calcula a média das médias, minimas, máximas e medianas:
media_mensal_sintetica = df_anosintetico.groupby('Mes')['Media'].mean().reset_index()
media_minima_sintetica = df_anosintetico.groupby('Mes')['Minima'].mean().reset_index()
media_maxima_sintetica = df_anosintetico.groupby('Mes')['Maxima'].mean().reset_index()
mediana_mensal_sintetica = df_anosintetico.groupby('Mes')['Mediana'].mean().reset_index()

# Dicionário para converter número do mês em nome
meses = {
    1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun',
    7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'
}
media_mensal_sintetica['MesNome'] = media_mensal_sintetica['Mes'].map(meses)
media_minima_sintetica['MesNome'] = media_mensal_sintetica['Mes'].map(meses)
media_maxima_sintetica['MesNome'] = media_mensal_sintetica['Mes'].map(meses)
mediana_mensal_sintetica['MesNome'] = mediana_mensal_sintetica['Mes'].map(meses)

# Ordena corretamente os meses
media_mensal_sintetica = media_mensal_sintetica.sort_values('Mes')


# Junta os três DataFrames em um só, usando 'Mes' como chave
df_plot = media_mensal_sintetica[['Mes', 'Media']].copy()
df_plot['Minima'] = media_minima_sintetica['Minima']
df_plot['Maxima'] = media_maxima_sintetica['Maxima']
df_plot['MesNome'] = df_plot['Mes'].map(meses)

# Ordena corretamente os meses
df_plot = df_plot.sort_values('Mes')

# Plotagem
plt.figure(figsize=(12, 6))

# Faixa entre mínima e máxima
plt.fill_between(
    df_plot['MesNome'],
    df_plot['Minima'],
    df_plot['Maxima'],
    color='lightblue',
    alpha=0.4,
    label='Faixa (Mínima - Máxima)'
)

# Linha da média
plt.plot(df_plot['MesNome'], df_plot['Media'], marker='o', color='blue', label='Média Mensal')

# Linha da mínima
plt.plot(df_plot['MesNome'], df_plot['Minima'], linestyle='--', color='green', label='Média Mínima')

# Linha da máxima
plt.plot(df_plot['MesNome'], df_plot['Maxima'], linestyle='--', color='red', label='Média Máxima')

# Ajustes do gráfico
plt.title('Ano Sintético da Vazão Média Mensal (com Mínima e Máxima)')
plt.xlabel('Mês')
plt.ylabel('Vazão')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#Plotando a média
plt.figure(figsize=(10, 5))

# Linha da média mensal
plt.plot(media_mensal_sintetica['MesNome'], media_mensal_sintetica['Media'],
         marker='o', color='blue', label='Média Mensal')

# Linha da mediana mensal
plt.plot(
    mediana_mensal_sintetica['MesNome'], mediana_mensal_sintetica['Mediana'],
    marker='s', color='red', linestyle='--', label='Mediana Mensal'
)

# Ajustes do gráfico
plt.title('Ano Sintético - Vazão Média Mensal')
plt.xlabel('Mês')
plt.ylabel('Vazão Média')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


# Plota o gráfico de XXX
df_ordenado = df_corrected
df_ordenado['Media'] = sorted(df_ordenado['Media'])
df_corrected.plot(x='Data', y='Media', kind='line')
plt.title('Curva de XXXX')
plt.xlabel('variable')
plt.ylabel('vazão')
plt.grid(True)
plt.show()


Grafico de Rippl
---
___

### Reservatório



In [4]:
# ============================================================
# CÉLULA — Balanço mensal do reservatório
# Vf = Vi + ΔV_Q, onde ΔV_Q = round(Q_in - Q_out, 3) * dias * 24 * 60 * 60
# Cria VR = max(Vf) ao final
# ============================================================

import os
import numpy as np
import pandas as pd

def _to_float(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float)): return float(x)
    return float(str(x).replace(',', '.'))

def _col(df, nome):
    alvo = nome.strip().lower()
    for k in df.columns:
        if k.strip().lower() == alvo:
            return k
    raise KeyError(f"Coluna obrigatória não encontrada: '{nome}'. Presentes: {list(df.columns)}")

# ---- entrada ----
ARQ = r"/content/PEE---2025/Reservatorio_1.xlsx"
if not os.path.exists(ARQ):
    raise FileNotFoundError(f"Arquivo não encontrado: {ARQ}")

df = pd.read_excel(ARQ)

# Normaliza números
for nome in ["Qsintética", "Qsaida"]:
    df[_col(df, nome)] = df[_col(df, nome)].map(_to_float)

# Dias (se não houver, usa calendário padrão)
dias_padrao = [31,28,31,30,31,30,31,31,30,31,30,31]
if any(cn.strip().lower() == "dias" for cn in df.columns):
    df["Dias"] = df[_col(df, "Dias")].map(_to_float)
else:
    df["Dias"] = pd.Series(dias_padrao[:len(df)])

# ---- balanço simples ----
CASAS_Q = 3  # arredondamento de Qliq (m³/s)
n = len(df)

Vi_list, Vf_list, Qliq_list, dV_Q_list = [], [], [], []

V_i = 0.0  # janeiro começa zerado
for i in range(n):
    Qin  = _to_float(df[_col(df, "Qsintética")].iloc[i])  # m³/s
    Qout = _to_float(df[_col(df, "Qsaida")].iloc[i])      # m³/s
    dias = float(df["Dias"].iloc[i])

    Qliq = round(Qin - Qout, CASAS_Q)               # m³/s
    dV_Q = Qliq * dias * 24 * 60 * 60               # m³
    V_f  = V_i + dV_Q                                # soma

    Vi_list.append(V_i);  Vf_list.append(V_f)
    Qliq_list.append(Qliq); dV_Q_list.append(dV_Q)

    V_i = V_f  # próximo mês começa em Vf

# ---- tabela final (somente balanço) ----
out = pd.DataFrame({
    "Mês": df[_col(df,"Mês")] if "Mês" in df.columns else np.arange(1, n+1),
    "Dias": df["Dias"].astype(int),
    "Q_in (m3/s)": df[_col(df,"Qsintética")].round(3),
    "Q_out (m3/s)": df[_col(df,"Qsaida")].round(3),
    "Qliq (m3/s)": np.array(Qliq_list).round(CASAS_Q),
    "ΔV_Q (m3)":   np.array(dV_Q_list).round(2),
    "Vi (m3)":     np.array(Vi_list).round(2),
    "Vf (m3)":     np.array(Vf_list).round(2),
})

# ---- VR: Volume do Reservatório = max(Vf) ----
VRu = float(out["Vf (m3)"].max() - out["Vf (m3)"].min() )
print(f"VRu (Volume Útil do Reservatório) = {VRu:,.2f} m³")

VRu (Volume Útil do Reservatório) = 54,341,366.40 m³
